In [1]:
import os
import numpy as np
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

# ========== 1. 配置路径 ==========
base_dir = r"D:/HuaweiMoveData/Users/hps_h/Desktop/CWRU"

# 定义 10 类（自己确认文件名和类别对应关系！）
label_dict = {
    "Normal": 0,
    "IR007": 1,
    "IR014": 2,
    "IR021": 3,
    "B007": 4,
    "B014": 5,
    "B021": 6,
    "OR007": 7,
    "OR014": 8,
    "OR021": 9
}

# ========== 2. 读取数据 ==========
def load_cwru_data(base_dir, segment_length=1024):
    X, y = [], []

    for root, _, files in os.walk(base_dir):
        for file in files:
            if file.endswith(".mat"):
                file_path = os.path.join(root, file)
                data = loadmat(file_path)

                # 取第一个 key（不同.mat可能 key 不一样，需要你确认，比如"XDE_time"）
                key = [k for k in data.keys() if "DE" in k or "DE_time" in k][0]
                signal = data[key].squeeze()

                # 分段
                num_segments = len(signal) // segment_length
                signal = signal[:num_segments * segment_length]
                segments = signal.reshape(num_segments, segment_length)

                # 确定类别
                label = None
                for k, v in label_dict.items():
                    if k in file:
                        label = v
                        break

                if label is not None:
                    X.append(segments)
                    y.extend([label] * num_segments)

    X = np.vstack(X)
    y = np.array(y)
    return X, y

X, y = load_cwru_data(base_dir)
print("数据形状:", X.shape, y.shape)

# ========== 3. 数据预处理 ==========
# 增加 channel 维度
X = X[..., np.newaxis]
y = to_categorical(y, num_classes=len(label_dict))

# 切分训练/测试集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("训练集:", X_train.shape, y_train.shape)
print("测试集:", X_test.shape, y_test.shape)

# ========== 4. 构建 1D CNN ==========
model = Sequential([
    Conv1D(16, kernel_size=32, activation='relu', input_shape=(X.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Conv1D(32, kernel_size=16, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(label_dict), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# ========== 5. 训练 ==========
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=10,
    batch_size=64
)

# ========== 6. 测试 ==========
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_acc)


数据形状: (34128, 1024) (34128,)
训练集: (27302, 1024, 1) (27302, 10)
测试集: (6826, 1024, 1) (6826, 10)


D:\study\anaconda\envs\tf2\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 993, 16)             │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 496, 16)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 481, 32)             │           8,224 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 240, 32)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 7680)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │         491,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │             650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 500,986 (1.91 MB)

 Trainable params: 500,986 (1.91 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
427/427 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.3978 - loss: 1.6090 - val_accuracy: 0.7479 - val_loss: 0.7348
Epoch 2/10
427/427 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.7034 - loss: 0.8181 - val_accuracy: 0.8293 - val_loss: 0.4911
Epoch 3/10
427/427 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.7718 - loss: 0.6201 - val_accuracy: 0.8758 - val_loss: 0.3886
Epoch 4/10
427/427 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - accuracy: 0.8108 - loss: 0.5155 - val_accuracy: 0.8957 - val_loss: 0.3159
Epoch 5/10
427/427 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.8355 - loss: 0.4524 - val_accuracy: 0.8933 - val_loss: 0.2900
Epoch 6/10
427/427 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - accuracy: 0.8571 - loss: 0.3929 - val_accuracy: 0.9266 - val_loss: 0.2192
Epoch 7/10
427/427 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.8755 - loss: 0.3393 - val_accuracy: 0.9273 - val_loss: 0.2058
Epoch 8/10
427/427 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - accuracy: 0.8838 - loss: 0.3129 - val_accu